# Tabela Silver Clientes

## Overview

 | Detail Tag | Information |
 |------------|-------------|
 |Versão inicial | Paulo Henrique  |
 |Tabela final | prata.clientes |
 |Origem dos dados |Camada Silver |


## Histórico de atualização
 | Date | Developed By | Reason |
 |:----:|--------------|--------|
 |30/01/2024 | Paulo Henrique |Criação do notebook |
 
 

## Informações

Este notebook contém a construção da tabela silver contendo os dados de clientes

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS prata;


In [0]:
%python
from pyspark.sql.functions import concat, lit, substring, regexp_replace, col, when

In [0]:
%sql
create or replace table prata.clientes as
select * from bronze.clientes

num_affected_rows,num_inserted_rows


In [0]:
%sql
UPDATE prata.clientes
SET Nome = REPLACE(REPLACE(REPLACE(REPLACE(Nome, 'Sr.', ''), 'Srta.', ''), 'Sra.', ''), 'Dr.', '')

num_affected_rows
200


In [0]:
%python
df=spark.table('prata.clientes')
df.show()

+----------+--------------------+--------------------+-------------------+----------+----------+
|id_cliente|                nome|               email|           telefone|data_carga|hora_carga|
+----------+--------------------+--------------------+-------------------+----------+----------+
|         1|Marcos Vinicius P...|moreirarafael@gma...|+55 (051) 1476-7634|2024-02-20|  16:22:21|
|         2|    Ana Laura Novaes|aliciapires@gmail...|   +55 71 2686-2429|2024-02-20|  16:22:21|
|         3|  Francisco Silveira| pteixeira@gmail.com|       31 6691-7704|2024-02-20|  16:22:21|
|         4|      Elisa da Rocha|enascimento@gmail...|       11 1007-1988|2024-02-20|  16:22:21|
|         5|        Kevin da Luz|moreirajoao-felip...|      0500-152-8310|2024-02-20|  16:22:21|
|         6|      Gabriela Cunha|alexandre58@gmail...|      0800 925 0073|2024-02-20|  16:22:21|
|         7|        Lucca Aragão|ana-carolina81@gm...|   +55 51 3202 5452|2024-02-20|  16:22:21|
|         8|    Vinicius Coste

In [0]:
%python


# Primeiro, remover '+55', os parênteses e espaços da coluna 'telefone'
df = df.withColumn("telefone", regexp_replace("telefone", "\+55|\(|\)|\s+", ""))

# Remover caracteres não numéricos para garantir que estamos partindo de uma base numérica limpa
df = df.withColumn("telefone", regexp_replace(col("telefone"), "[^0-9]", ""))

# Remover zeros à esquerda
df = df.withColumn("telefone", regexp_replace(col("telefone"), "^0+", ""))

# Substituir registros que comecem com 100, 200, 300, 400, 500, 600, 700, 800, 900 por '99 99999999'
df = df.withColumn("telefone", when(col("telefone").rlike("^(100|200|300|400|500|600|700|800|900)"), "99 99999999").otherwise(col("telefone")))

# Inserir um espaço após os dois primeiros números
df = df.withColumn("telefone", regexp_replace(col("telefone"), "^(\\d{2})(\\d+)", "$1 $2"))
df.show()




+----------+--------------------+--------------------+-----------+----------+----------+
|id_cliente|                nome|               email|   telefone|data_carga|hora_carga|
+----------+--------------------+--------------------+-----------+----------+----------+
|         1|Marcos Vinicius P...|moreirarafael@gma...|51 14767634|2024-02-20|  16:22:21|
|         2|    Ana Laura Novaes|aliciapires@gmail...|71 26862429|2024-02-20|  16:22:21|
|         3|  Francisco Silveira| pteixeira@gmail.com|31 66917704|2024-02-20|  16:22:21|
|         4|      Elisa da Rocha|enascimento@gmail...|11 10071988|2024-02-20|  16:22:21|
|         5|        Kevin da Luz|moreirajoao-felip...|99 99999999|2024-02-20|  16:22:21|
|         6|      Gabriela Cunha|alexandre58@gmail...|99 99999999|2024-02-20|  16:22:21|
|         7|        Lucca Aragão|ana-carolina81@gm...|51 32025452|2024-02-20|  16:22:21|
|         8|    Vinicius Costela|maysaporto@gmail.com|99 99999999|2024-02-20|  16:22:21|
|         9|       Ca

In [0]:
%python
df.write.format('delta').mode('overwrite').option('overwriteSchema', True).saveAsTable(f'prata.clientes')

In [0]:
%sql
select * from prata.clientes

id_cliente,nome,email,telefone,data_carga,hora_carga
1,Marcos Vinicius Pires,moreirarafael@gmail.com,51 14767634,2024-02-20,16:22:21
2,Ana Laura Novaes,aliciapires@gmail.com,71 26862429,2024-02-20,16:22:21
3,Francisco Silveira,pteixeira@gmail.com,31 66917704,2024-02-20,16:22:21
4,Elisa da Rocha,enascimento@gmail.com,11 10071988,2024-02-20,16:22:21
5,Kevin da Luz,moreirajoao-felipe@gmail.com,99 99999999,2024-02-20,16:22:21
6,Gabriela Cunha,alexandre58@gmail.com,99 99999999,2024-02-20,16:22:21
7,Lucca Aragão,ana-carolina81@gmail.com,51 32025452,2024-02-20,16:22:21
8,Vinicius Costela,maysaporto@gmail.com,99 99999999,2024-02-20,16:22:21
9,Cauê Teixeira,marcelocostela@gmail.com,84 35150712,2024-02-20,16:22:21
10,Vitor Hugo Barbosa,xnunes@gmail.com,41 11684578,2024-02-20,16:22:21
